In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from executorch.backends.xnnpack.partition.xnnpack_partitioner import XnnpackPartitioner
from executorch.exir import to_edge_transform_and_lower
from executorch.runtime import Runtime

In [6]:
CLASSES = ["biological", "trash", "metal", "plastic", "battery", "glass", "paper", "clothes", "cardboard", "shoes"]

In [2]:
student = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.IMAGENET1K_V2)

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to /Users/dawidwozniak/.cache/torch/hub/checkpoints/mobilenet_v3_large-5c1a4163.pth


100%|██████████| 21.1M/21.1M [00:00<00:00, 30.4MB/s]


In [3]:
student.classifier

Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1280, out_features=1000, bias=True)
)

In [14]:
student(torch.randn(1, 3, 128, 128))

tensor([[ 0.1243, -0.1683,  0.1040,  0.1607, -0.1002,  0.0401,  0.2072,  0.1326,
          0.0241, -0.0797]], grad_fn=<AddmmBackward0>)

In [7]:
student.classifier[3] = nn.Linear(1280, len(CLASSES))

In [ ]:
# 1. Export your PyTorch model
model = student.eval()
example_inputs = (torch.randn(1, 3, 254, 254),)
exported_program = torch.export.export(model, example_inputs)

# 2. Optimize for target hardware (switch backends with one line)
program = to_edge_transform_and_lower(
    exported_program,
    partitioner=[XnnpackPartitioner()],  # CPU | CoreMLPartitioner() for iOS | QnnPartitioner() for Qualcomm
).to_executorch()

# 3. Save for deployment
with open("model.pte", "wb") as f:
    f.write(program.buffer)

# Test locally via ExecuTorch runtime's pybind API (optional)

runtime = Runtime.get()
method = runtime.load_program("model.pte").load_method("forward")
outputs = method.execute([torch.randn(1, 3, 254, 254)])
print(outputs)

[program.cpp:154] InternalConsistency verification requested but not available


[tensor([[-0.0964, -0.1225,  0.1680, -0.2781, -0.1837,  0.0143, -0.2732, -0.0821,
          0.1349, -0.0332]])]
